In [1]:
import librosa as lr
import librosa.display
import numpy as np

import extract_features
import utilty

ipd.display(ipd.HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# %%timeit
track = utilty.get_audio_tracks()[0]
track = utilty.find_file('without')

sec = 30
duration = utilty.get_duration(track)
segments = int(duration) // sec
audio_30secs = [lr.load(track, offset=sec*i, duration=sec) for i in range(segments)]
name = track.stem.split('-')[-1].replace('_', ' ').title()
track

PosixPath('/Users/matteding/Desktop/Whabam/tracks/eminem-2002-the_eminem_show-10-without_me.mp3')

In [3]:
# all features with shape (1, t)
feature_funcs = [lr.feature.rmse, lr.feature.spectral_bandwidth, lr.feature.spectral_centroid, 
                 lr.feature.spectral_flatness, lr.feature.spectral_rolloff, lr.feature.zero_crossing_rate]
aggs = []
for audio, _ in audio_30secs:
    segment = dict()
    for func in feature_funcs:
        result = func(audio)
        agg = utilty.get_aggregations(result)
        segment[func.__name__] = agg
    aggs.append(segment)

In [16]:
aggs[-1]['spectral_flatness']

Aggregations(mean=0.021195393, median=0.0025111442, mode=4.933534e-07, amax=0.1937035, amin=4.933534e-07, std=0.039248634, sum=27.384447)

In [17]:
extract_features.misc_feature_aggs(audio)['spectral_flatness']

Aggregations(mean=0.021195393, median=0.0025111442, mode=4.933534e-07, amax=0.1937035, amin=4.933534e-07, std=0.039248634, sum=27.384447)

In [5]:
tonnetzes = [lr.feature.tonnetz(audio) for audio, _ in audio_30secs]
tonnetz_aggs = [utilty.get_aggregations(tonnetz, axis=1) for tonnetz in tonnetzes]

In [6]:
tonnetz_aggs[0].mean.shape # 6 for perfect 5ths, major 3rds, minor 3rds with x and y

(6,)

In [19]:
tnz = tonnetz_aggs[-1]

In [20]:
tnz

Aggregations(mean=array([ 0.01428755,  0.01690418,  0.14570838, -0.0501279 ,  0.02151135,
       -0.01199497]), median=array([ 0.0138555 ,  0.01652153,  0.14819762, -0.02714751,  0.02441909,
       -0.01689105]), mode=array([-0.09310386, -0.05089711, -0.17309714, -0.40716162, -0.08455134,
       -0.08348381]), amax=array([0.13017412, 0.10159274, 0.4034588 , 0.09436358, 0.10272051,
       0.09731782]), amin=array([-0.09310386, -0.05089711, -0.17309714, -0.40716162, -0.08455134,
       -0.08348381]), std=array([0.02753574, 0.02159649, 0.12742599, 0.09356664, 0.04020301,
       0.02847409]), sum=array([ 18.45950844,  21.84019646, 188.25522894, -64.7652491 ,
        27.79266788, -15.49750483]))

In [18]:
extract_features.tonnetz_aggs(audio)['x_P5']

Aggregations(mean=0.014287545230310792, median=0.013855497420920328, mode=-0.09310386465856413, amax=0.1301741226093064, amin=-0.09310386465856413, std=0.027535742981675205, sum=18.459508437561542)

In [14]:
from importlib import reload
reload(utilty)
reload(extract_features);